In [1]:
import csv
import numpy as np
from pprint import pprint
from sturdr.nav.ephemeris import GetNavStates
from sturdr.nav.estimation import LeastSquares, LeastSquaresPos
from sturdr.utils.constants import LIGHT_SPEED, GPS_L1CA_CARRIER_FREQ, GPS_L1CA_CODE_FREQ
from sturdr.utils.coordinates import ecef2lla, eci2ecef
from sturdr.dsp.discriminator import DllVariance

BETA = LIGHT_SPEED / GPS_L1CA_CODE_FREQ
LAMBDA = LIGHT_SPEED / GPS_L1CA_CARRIER_FREQ

# PRN = [1, 7, 14, 17, 19, 21, 30]
CNo = 10**(np.asarray([36.826, 41.229, 43.359, 36.599, 36.312, 43.221, 42.617])/10)
ToW = 413660.94
doppler = np.array([1790.1266367785295, -474.3815058289274, 3587.6279305533444, 3279.2840787000678, 5023.270101382448, -341.27711577144595, 1520.6863470836226])
code_phase = np.array([11425.047804671016, 8208.201776750902, 11330.930299839343, 10909.063374637126, 3964.2406759900205, 4235.0250637613835, 12225.85874056617])
code_phase_time = code_phase / GPS_L1CA_CODE_FREQ
transmit_time = ToW + code_phase_time
receive_time = transmit_time.max() + 0.068802 # common receive time/sample including transmit time
tgd = np.zeros(7)
ephem = {}

with open("../results/GPS_L1CA_TEST_Ephemeris.csv", "r") as file:
    reader = csv.DictReader(file)
    for i,item in enumerate(reader):
        item.pop('id', None)
        item.pop('iode', None)
        item.pop('iodc', None)
        item.pop('ura', None)
        item.pop('health', None)
        for k,v in item.items():
            item[k] = np.float64(v)
        item["transmit_time"] = transmit_time[i]
        ephem[i] = item
# pprint(ephem[0])
# pprint(ephem[0].keys())

pprint(code_phase_time)
pprint(transmit_time)

array([0.01116818, 0.00802366, 0.01107618, 0.0106638 , 0.00387511,
       0.00413981, 0.01195099])
array([413660.95116818, 413660.94802366, 413660.95107618, 413660.9506638 ,
       413660.94387511, 413660.94413981, 413660.95195099])


In [2]:
sv_pos = np.zeros((7,3))
# sv_pos_new = np.zeros((7,3))
sv_vel = np.zeros((7,3))
sv_clk = np.zeros((7,3))

# get satellite positions based on transmit time
for i in range(7):
    sv_clk[i,:], sv_pos[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    # sv_pos_new[i,:] = CorrectEarthRotation(-(receive_time - transmit_time[i]), sv_pos[i,:])
    tgd[i] = ephem[i]['tgd']
# pprint(sv_pos - sv_pos_new)

psr = (receive_time - transmit_time + sv_clk[:,0] + tgd) * LIGHT_SPEED
psrdot = -LAMBDA * doppler + sv_clk[:,1] * LIGHT_SPEED

# get corrected positions with new transmit time
# new_transmit_time = receive_time - psr
# for i in range(7):
    # ephem[i]['transmit_time'] = new_transmit_time[i]
    # sv_clk[i,:], sv_pos_new[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    # sv_pos_new[i,:] = eci2ecef(+psr[i], sv_pos_new[i,:])
    # sv_pos_new[i,:] = eci2ecef(+psr[i], sv_pos[i,:])

# pprint(transmit_time - new_transmit_time)
# pprint(sv_pos - sv_pos_new)

In [4]:
R = np.diag(BETA**2 * DllVariance(CNo, 0.02))
x, _, _ = LeastSquaresPos(sv_pos, psr, R, np.zeros(4))
x1, _ = LeastSquares(sv_pos, sv_vel, psr, psrdot, CNo, np.zeros(8))
true_x = np.asarray([422596.629, -5362864.287, 3415493.797])
true_range = np.linalg.norm(true_x[...] - sv_pos, axis=1)

np.set_printoptions(suppress=True)
print(f"lla_est = {np.array2string(ecef2lla(x[:3]), precision=8)}")
print(f"x_est   = {np.array2string(x, precision=3)}")
print(f"x1_est  = {np.array2string(x1[[0,1,2,6]], precision=3)}")
print(f"x_true  = {np.array2string(true_x, precision=3)}")
print(f"x1_vel  = {np.array2string(x1[[3,4,5,7]], precision=3)}")
print(f"diff    = {np.array2string(true_x - x[:3], precision=3)}, norm = {np.array2string(np.linalg.norm(true_x - x[:3]), precision=6)}")
print(f"diff1   = {np.array2string(true_x - x1[:3], precision=3)}, norm = {np.array2string(np.linalg.norm(true_x - x1[:3]), precision=6)}")
print(f"psr     = {np.array2string(psr, precision=0)}")
print(f"r_true  = {np.array2string(true_range, precision=0)}")
print(f"diff    = {np.array2string(true_range - psr, precision=3)}")

lla_est = [ 32.58631234 -85.49442298 198.7709868 ]
x_est   = [  422591.95  -5362865.971  3415499.054    67785.7  ]
x1_est  = [  422591.95  -5362865.971  3415499.054    67785.7  ]
x_true  = [  422596.629 -5362864.287  3415493.797]
x1_vel  = [   0.008   -0.006   -0.01  -471.156]
diff    = [ 4.679  1.684 -5.257], norm = 7.236322
diff1   = [ 4.679  1.684 -5.257], norm = 7.236322
psr     = [20921613. 21870201. 20887921. 21223797. 23134615. 23014302. 20468445.]
r_true  = [20853843. 21802422. 20820137. 21156004. 23066809. 22946537. 20400653.]
diff    = [-67769.42  -67779.035 -67783.307 -67793.725 -67806.355 -67765.198
 -67791.216]
